In [1]:
import intera_interface
import rospy
import json
import tf
from copy import deepcopy
from sawyer import transformations
from os import environ

In [2]:
environ['ROS_MASTER_URI'] = 'http://021607CP00116.local:11311'

In [3]:
environ['ROS_HOSTNAME'] = 'mingew.local'

In [4]:
rospy.init_node("sawyer_example")

In [5]:
effector = "right_hand"
world = "world"

In [6]:
tfl = tf.TransformListener(interpolate=True, cache_time=rospy.Duration(1000))
tfb = tf.TransformBroadcaster()

In [7]:
rs = intera_interface.RobotEnable(intera_interface.CHECK_VERSION)

In [8]:
rs.enable()

[INFO] [1555256895.829613]: Robot Enabled


In [9]:
limb = intera_interface.Limb('right')

In [10]:
hope_lower_left = {u'right_j0': 1.611796875,
                   u'right_j1': -2.75679296875,
                   u'right_j2': -3.0216201171875,
                   u'right_j3': 1.296830078125,
                   u'right_j4': 1.161048828125,
                   u'right_j5': -1.98382421875,
                   u'right_j6': 1.0612880859375}

In [25]:
limb.move_to_neutral()

In [24]:
limb.move_to_joint_positions(hope_lower_left)
# Move to extreme lower left in JOINT space

# Compute sticks in cartesian space

In [13]:
# Base is ["lower"]["left"][0] for HOPE
base_position = [-0.5, -0.7, 0.35]


In [14]:
# Starting from lower left (drawing direction)
# 
sticks = {"hope": [], "fear": []}
y_interval = 0.01     # Interval of sticks in y direction
z_interval = 0.02      # Interval of sticks in z direction
group_interval = 0.01  # Interval between each group in y direction
num_groups = 8         # Number of stick groups per line
num_sticks = 4         # Number of sticks per group
height_stick = 0.03    # Length in z direction
num_height = 8         # Number of sticks in z direction (in height)
num_points_cart = 20   # Number of cartesian points for the actual drawing motion

emotion = "fear"
z_stick = base_position[2]
col = 0
for h in range(num_height):
    y_stick = base_position[1]
    for g in range(num_groups):
        for i in range(num_sticks):
            # Add the end stick
            end_stick = [base_position[0], y_stick, z_stick + height_stick]
            tfb.sendTransform(end_stick, [0, 0, 0, 1], rospy.Time.now(), str(col), "world")
            motion = [end_stick]

            # Add the cartesian motion
            for p in range(num_points_cart):
                motion_point = deepcopy(end_stick)
                motion_point[2] = z_stick + height_stick - height_stick * (p+1) / num_points_cart
                motion.append(motion_point)
                #if i == 0 and g == 0:
                #    tfb.sendTransform(motion_point, [0, 0, 0, 1], rospy.Time.now(), str(col)+'-'+str(p), "world")
            
            sticks[emotion].append(motion)
            y_stick += y_interval
            col += 1
        y_stick += group_interval
    z_stick += z_interval + height_stick
    if len(sticks["fear"]) == num_groups*num_height*num_sticks/2:
        emotion = "hope"
        z_stick += 0.02
print(len(sticks["hope"]), "+", len(sticks["fear"]), "sticks generated")

(128, '+', 128, 'sticks generated')


In [15]:
sticks['fear'][0]

[[-0.5, -0.7, 0.38],
 [-0.5, -0.7, 0.3785],
 [-0.5, -0.7, 0.377],
 [-0.5, -0.7, 0.3755],
 [-0.5, -0.7, 0.374],
 [-0.5, -0.7, 0.3725],
 [-0.5, -0.7, 0.371],
 [-0.5, -0.7, 0.3695],
 [-0.5, -0.7, 0.368],
 [-0.5, -0.7, 0.3665],
 [-0.5, -0.7, 0.365],
 [-0.5, -0.7, 0.3635],
 [-0.5, -0.7, 0.362],
 [-0.5, -0.7, 0.3605],
 [-0.5, -0.7, 0.359],
 [-0.5, -0.7, 0.3575],
 [-0.5, -0.7, 0.356],
 [-0.5, -0.7, 0.3545],
 [-0.5, -0.7, 0.353],
 [-0.5, -0.7, 0.35150000000000003],
 [-0.5, -0.7, 0.35]]

# Cartesian sticks

In [16]:
hand_T_stick = [[0.14239612794977008, 0.01701314337065485, -0.07630844138995452],
 [0.5270677868272005,
  0.023880201241985057,
  -0.8461224120032248,
  0.07553904939543005]]

In [17]:
stick_fk = {"hope" : [], "fear": [], "joints": []}
seed = None
for emotion in ["hope", "fear"]:
    for stick_id, stick in enumerate(sticks[emotion]):
        motion = []
        for point_id, point in enumerate(stick):
            hand_T_world = transformations.multiply_transform([point, [0,0,0,1]], hand_T_stick)
            tfb.sendTransform(hand_T_world[0], hand_T_world[1], rospy.Time.now(), "eik", world)
            result = limb.ik_request(transformations.list_to_pose(hand_T_world), joint_seed=seed)
            if result == False:
                raise RuntimeError(emotion + ": can't compute stick start " + str(stick_id))
            else:
                motion.append(result.values())
                seed = result
        stick_fk[emotion].append(motion)
        stick_fk["joints"] = result.keys()
        #rospy.sleep(0.1)

In [18]:
stick_fk["hope"][0]

[[0.6436005340632613,
  -1.781924030201659,
  1.0553534111880427,
  1.0935538435786258,
  -3.0226,
  -2.5198718909283784,
  1.6007634816129355],
 [0.6454539255927305,
  -1.7832841260348669,
  1.0548429237998695,
  1.0944443888647413,
  -3.021673411288561,
  -2.5212991125272755,
  1.6001432357347138],
 [0.6473232367833698,
  -1.7846391435913882,
  1.054363193995599,
  1.0953439237305493,
  -3.020682392244713,
  -2.5227265831710377,
  1.599544560113151],
 [0.6491891099710086,
  -1.785991500322399,
  1.0538898046697147,
  1.0962313833972617,
  -3.0197000254690605,
  -2.5241621394516516,
  1.5989511001329375],
 [0.6510537814499933,
  -1.7873389752015547,
  1.0534209363871128,
  1.0971075536104244,
  -3.0187256448594715,
  -2.525605176999914,
  1.5983621539542974],
 [0.6529150798763935,
  -1.7886837196543204,
  1.0529581945322855,
  1.097971866394908,
  -3.017759731620502,
  -2.5270561023484377,
  1.5977783627889672],
 [0.6547751117463279,
  -1.7900237167281,
  1.0525000160849978,
  1.09882

In [19]:
from intera_motion_interface import (
    MotionTrajectory,
    MotionWaypoint,
    MotionWaypointOptions
)

In [20]:
traj  = MotionTrajectory(limb=limb)

In [21]:
traj.set_joint_names(stick_fk["joints"])

In [22]:
for point in stick_fk["hope"][0]:
    waypoint = MotionWaypoint(limb=limb)
    waypoint.set_joint_angles(point)
    traj.append_waypoint(waypoint)

In [23]:
traj.send_trajectory(timeout=3)

In [36]:
traj.to_msg()

label: "default"
joint_names: [right_j6, right_j5, right_j4, right_j3, right_j2, right_j1, right_j0]
waypoints: 
  - 
    joint_positions: [0.6436018239776315, -1.7819240856192806, 1.0553549767211, 1.0935556244362665, -3.0226, -2.5198711311482422, 1.600764467777977]
    active_endpoint: "right_hand"
    pose: 
      header: 
        seq: 0
        stamp: 
          secs: 0
          nsecs:         0
        frame_id: ''
      pose: 
        position: 
          x: 0.0
          y: 0.0
          z: 0.0
        orientation: 
          x: 0.0
          y: 0.0
          z: 0.0
          w: 0.0
    options: 
      label: "default"
      max_joint_speed_ratio: 0.7
      joint_tolerances: []
      max_joint_accel: [7.0, 5.0, 8.0, 8.0, 8.0, 8.0, 8.0]
      max_linear_speed: 0.6
      max_linear_accel: 0.6
      max_rotational_speed: 1.57
      max_rotational_accel: 1.57
      corner_distance: 0.0
  - 
    joint_positions: [0.6454549056753465, -1.7832841654372227, 1.0548441126969956, 1.09444573

# Overwrite other poses

In [ ]:
# Pause pose
poses = {"pause": limb.joint_angles()}

with open("../config/poses.json", "w") as f:
    json.dump(poses, f, indent=4)

In [17]:
with open("../config/poses.json") as f:
    poses = json.load(f)

In [ ]:
limb.move_to_joint_positions(poses["pause"])

In [18]:
poses

{u'pause': {u'right_j0': 2.462169921875,
  u'right_j1': -3.327517578125,
  u'right_j2': -2.2095576171875,
  u'right_j3': -1.52369140625,
  u'right_j4': 0.1305107421875,
  u'right_j5': -1.00734375,
  u'right_j6': -0.2455576171875}}

# Display sticks

In [ ]:
with open("../config/sticks_target_ik.json") as f:
    sticks = json.load(f)

In [ ]:
from sensor_msgs.msg import JointState
p = rospy.Publisher("display/joint_states", JointState, queue_size=1)

In [ ]:
type = 'fear'
side = 'end'
for i, pose in enumerate(sticks[type]):
    js = JointState()
    js.header.stamp = rospy.Time.now()
    js.name = pose[side].keys()
    js.position = pose[side].values()
    p.publish(js)
    input = raw_input('Press to display {} pose {} of [q to quit] '.format(side, i, type))
    if input == 'q': break

In [ ]:
sticks